In [1]:
from ultralytics import YOLO
import os
import cv2
import shutil
from sklearn.model_selection import train_test_split
import numpy as np

In [10]:
IMAGES_DIR = 'soiling_dataset/images/train'  # Путь к вашему датасету с изображениями
MASKS_DIR = 'soiling_dataset/masks/train'  # Путь к вашему датасету с масками
OUTPUT_DIR = '../soiling_train'  # Путь к выходной директории
TRAIN_SIZE = 0.8  # Процент обучающей выборки

In [11]:
os.makedirs(os.path.join(OUTPUT_DIR, 'images/train'), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, 'images/val'), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, 'labels/train'), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, 'labels/val'), exist_ok=True)

In [12]:
image_files = [f for f in os.listdir(IMAGES_DIR) if f.lower().endswith(('.jpg', '.png'))]
mask_files = [f for f in os.listdir(MASKS_DIR) if f.endswith('.png')]
if len(image_files) != len(mask_files):
    print(len(image_files))
    print(len(mask_files))
    print("Количество изображений и масок не совпадает.")
    for mask in mask_files:
        if mask.replace(".png", ".jpg") not in image_files:
            print(mask)

In [17]:
from tqdm.auto import tqdm

In [14]:
train_images, val_images = train_test_split(image_files, train_size=TRAIN_SIZE, random_state=42)

In [18]:
def convert_mask_to_yolo(mask_path, out):
    # Открываем изображение
    image = cv2.imread(mask_path)

    if image is None:
        print(f"Не удалось открыть изображение: {mask_path}")
        return
    
    height, width = image.shape[:2]

    # Создаем маску для черного цвета
    black_mask = cv2.inRange(image, (0, 0, 0), (50, 50, 50))

    # Создаем новое изображение, где черный цвет остается, а остальные цвета становятся белыми
    new_image = np.ones_like(image) * 255  # Начинаем с белого изображения
    new_image[black_mask > 0] = [0, 0, 0]  # Заменяем черные пиксели

    # Преобразуем в градации серого для нахождения контуров
    gray_image = cv2.cvtColor(new_image, cv2.COLOR_BGR2GRAY)

    # Находим контуры
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Сохраняем контуры в текстовом формате
    output_file_name = os.path.splitext(os.path.basename(mask_path))[0] + '.txt'
    output_file_path = os.path.join(OUTPUT_DIR, out, output_file_name)

    with open(output_file_path, 'w') as f:
        for index, contour in enumerate(contours):
            # Получаем координаты всех точек контура
            contour_points = contour.reshape(-1, 2)
            # Нормализуем координаты
            normalized_points = [(x / width, y / height) for x, y in contour_points]
            points_str = ' '.join(f"{x:.3f} {y:.3f}" for x, y in normalized_points)
            f.write(f"0 {points_str}\n")

In [20]:
# Копирование изображений и масок в соответствующие папки
for img in tqdm(train_images):
    shutil.copy(os.path.join(IMAGES_DIR, img), os.path.join(OUTPUT_DIR, 'images/train', img))
    mask_name = img.replace('.jpg', '.png')
    convert_mask_to_yolo(os.path.join(MASKS_DIR, mask_name), 'labels/train')

  0%|          | 0/3200 [00:00<?, ?it/s]

In [21]:
for img in tqdm(val_images):
    shutil.copy(os.path.join(IMAGES_DIR, img), os.path.join(OUTPUT_DIR, 'images/val', img))
    mask_name = img.replace('.jpg', '.png')
    convert_mask_to_yolo(os.path.join(MASKS_DIR, mask_name), 'labels/val')


  0%|          | 0/800 [00:00<?, ?it/s]

In [22]:
data_yaml_content = f"""
train: soiling_train/images/train
val: soiling_train/images/val

nc: 1  # Обновите количество классов (1 для загрязнения)
names: ['contaminated']  # Обновите названия классов
"""

with open('data.yaml', 'w') as f:
    f.write(data_yaml_content)

print("Датасет успешно разбит и сохранен в структуре проекта.")

Датасет успешно разбит и сохранен в структуре проекта.


In [23]:
model = YOLO("yolo11s-seg.pt")

In [26]:
train_results = model.train(
    data="./data.yaml",  # path to dataset YAML
    imgsz=480,  # training image size
    batch = -1,
    cache = 'ram'
)

New https://pypi.org/project/ultralytics/8.3.47 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.44 🚀 Python-3.10.15 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11003MiB)
engine/trainer: task=segment, mode=train, model=yolo11s-seg.pt, data=./data.yaml, epochs=100, time=None, patience=100, batch=-1, imgsz=480, save=True, save_period=-1, cache=ram, device=None, workers=8, project=None, name=train6, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save


train: Scanning /home/glinsai/Projects/2024_Nornickel/2 задача/train_dataset/bas

WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


train: Caching images (1.5GB RAM): 100%|██████████| 3200/3200 [00:11<00:00, 277.

AutoBatch: Computing optimal batch size for imgsz=480 at 60.0% CUDA memory utilization.
AutoBatch: CUDA:0 (NVIDIA GeForce RTX 2080 Ti) 10.75G total, 4.80G reserved, 4.30G allocated, 1.64G free
      Params      GFLOPs  GPU_mem (GB)  forward (ms) backward (ms)                   input                  output


...


                 Class     Images  Instances      Box(P          R      mAP50  m

                   all        800       1067      0.986      0.974       0.99      0.905       0.98      0.972      0.988      0.851



100 epochs completed in 0.860 hours.
Optimizer stripped from runs/segment/train6/weights/last.pt, 20.5MB
Optimizer stripped from runs/segment/train6/weights/best.pt, 20.5MB

Validating runs/segment/train6/weights/best.pt...
Ultralytics 8.3.44 🚀 Python-3.10.15 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11003MiB)
YOLO11s-seg summary (fused): 265 layers, 10,067,203 parameters, 0 gradients, 35.3 GFLOPs



                Class     Images  Instances      Box(P          R      mAP50  m

                   all        800       1067      0.986      0.974      0.991      0.905      0.981      0.972      0.988      0.851
Speed: 0.1ms preprocess, 1.1ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to runs/segment/train6


In [ ]:
results = model.val(data="./data.yaml")
print(results)


In [26]:
def infer_image(image_path):
    # Загрузка изображения
    image = cv2.imread(image_path)

    # Инференс
    return model(image)


In [27]:
# Функция для создания маски с черным фоном
def create_mask(image_path, results):
    # Загружаем изображение и переводим в градации серого
    image = cv2.imread(image_path)
    height, width = image.shape[:2]

    # Создаем пустую маску с черным фоном
    mask = np.zeros((height, width), dtype=np.uint8)

    # Проходим по результатам и создаем маску
    for result in results:
        masks = result.masks  # Получаем маски из результатов
        if masks is not None:
            for mask_array in masks.data:  # Получаем маски как массивы
                mask_i = mask_array.numpy()  # Преобразуем маску в numpy массив
                
                # Изменяем размер маски под размер оригинального изображения
                mask_i_resized = cv2.resize(mask_i, (width, height), interpolation=cv2.INTER_LINEAR)
                
                # Накладываем маску на пустую маску (255 для белого)
                mask[mask_i_resized > 0] = 255

    return mask

In [ ]:
model(filename)

In [39]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolo11n-seg.pt")  # load an official model
model = YOLO("baseline.pt")  # load a custom model
model.cpu()
# Predict with the model
results = model(filename)  # predict on an image


image 1/1 /home/glinsai/Projects/2024_Nornickel/2 задача/train_dataset/baseline/datasets/train_data/images/val/1709807287_0.jpg: 384x640 2 contaminateds, 85.8ms
Speed: 1.4ms preprocess, 85.8ms inference, 2.8ms postprocess per image at shape (1, 3, 384, 640)


In [45]:
from ultralytics import YOLO

# Load a YOLOv8n PyTorch model
model = YOLO("b.pt")

# Export the model
model.export(format="openvino",task = 'segment')  # creates 'yolov8n_openvino_model/'

# Load the exported OpenVINO model
ov_model = YOLO("yolov8n_openvino_model/",task = 'segment')


Ultralytics 8.3.44 🚀 Python-3.10.15 torch-2.5.1+cu124 CPU (AMD Ryzen 5 3600 6-Core Processor)
YOLO11n-seg summary (fused): 265 layers, 2,868,664 parameters, 0 gradients, 10.4 GFLOPs

PyTorch: starting from 'yolo11n-seg.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) ((1, 116, 8400), (1, 32, 160, 160)) (5.9 MB)

OpenVINO: starting export with openvino 2024.5.0-17288-7975fa5da0c-refs/pull/3856/head...
OpenVINO: export success ✅ 4.3s, saved as 'yolo11n-seg_openvino_model/' (11.4 MB)

Export complete (4.6s)
Results saved to /home/glinsai/Projects/2024_Nornickel/2 задача/train_dataset/baseline
Predict:         yolo predict task=segment model=yolo11n-seg_openvino_model imgsz=640  
Validate:        yolo val task=segment model=yolo11n-seg_openvino_model imgsz=640 data=/ultralytics/ultralytics/cfg/datasets/coco.yaml  
Visualize:       https://netron.app


In [37]:
infer_image(filename)


0: 384x640 2 contaminateds, 246.4ms
Speed: 1.4ms preprocess, 246.4ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: ultralytics.engine.results.Masks object
 names: {0: 'contaminated'}
 obb: None
 orig_img: array([[[125, 140, 119],
         [125, 140, 119],
         [125, 140, 119],
         ...,
         [ 27,  34,  37],
         [ 27,  34,  37],
         [ 27,  34,  37]],
 
        [[124, 139, 118],
         [125, 140, 119],
         [125, 140, 119],
         ...,
         [ 27,  34,  37],
         [ 27,  34,  37],
         [ 27,  34,  37]],
 
        [[121, 138, 117],
         [123, 140, 119],
         [126, 141, 120],
         ...,
         [ 26,  33,  36],
         [ 27,  34,  37],
         [ 27,  34,  37]],
 
        ...,
 
        [[ 64,  77,  61],
         [ 64,  77,  61],
         [ 64,  78,  60],
         ...,
         [ 34,  32,  32],
         [ 34,  32,  32],
         [ 37,  35,  35]],
 
        [[ 64,  77,  61],
         [ 64,  77,  61],
         [ 64,  7

In [35]:
filename  = './datasets/train_data/images/val/1709807287_0.jpg'
results = infer_image(filename)
mask_image = create_mask(filename, results)

# Сохраняем маску в формате PNG
mask_output_path = './mask_image.png'  # Укажите путь для сохранения маски
cv2.imwrite(mask_output_path, mask_image)


0: 384x640 2 contaminateds, 259.1ms
Speed: 3.1ms preprocess, 259.1ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)


True

In [24]:
results

[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'contaminated'}
 obb: None
 orig_img: array([[[53, 35, 28],
         [53, 35, 28],
         [52, 34, 27],
         ...,
         [34, 41, 34],
         [35, 42, 35],
         [33, 40, 33]],
 
        [[53, 35, 28],
         [53, 35, 28],
         [52, 34, 27],
         ...,
         [60, 67, 60],
         [63, 70, 63],
         [64, 71, 64]],
 
        [[53, 35, 28],
         [53, 35, 28],
         [52, 34, 27],
         ...,
         [69, 76, 69],
         [75, 82, 75],
         [77, 84, 77]],
 
        ...,
 
        [[32, 26, 27],
         [32, 26, 27],
         [32, 26, 27],
         ...,
         [51, 56, 54],
         [44, 49, 47],
         [42, 47, 45]],
 
        [[29, 23, 24],
         [29, 23, 24],
         [29, 23, 24],
         ...,
         [43, 48, 46],
         [37, 42, 40],
         [33, 38, 36]],
 
        [[27, 21, 22]